In [443]:
!pip install xgboost==1.7.6
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import col, abs
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, GBTClassificationModel, ClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import when
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lit
from pathlib import Path
from pyspark import StorageLevel
from xgboost.spark import SparkXGBClassifier
import uuid
import warnings
import pandas as pd
import time
from datetime import timedelta
warnings.filterwarnings('ignore')

In [444]:
spark = SparkSession.builder.appName("ChurnXGBoost").master("local[*]").getOrCreate()
cores = spark.sparkContext.defaultParallelism 
target_shuffle_partitions = cores * 2 # set custom partitions for shuffling based on available cores
spark.conf.set("spark.sql.shuffle.partitions", target_shuffle_partitions) # will use above coun during shuffle operations
print("spark.sql.shuffle.partitions ->", spark.conf.get("spark.sql.shuffle.partitions")) # displays 

spark.sql.shuffle.partitions -> 24


In [445]:
def concatenate_clean_churn_files() -> DataFrame | None:
    """
    Combines all 'churn_clean*.csv' files in the current directory into a single Spark DataFrame.
    The first matching file determines the schema. All subsequent files are appended.
    If no matching files are found, the function returns None.
    Returns:
        churnCombinedDf: Spark DataFrame containing the combined data, or None if no files are found.
    """
    csvFiles = [str(file) for file in Path(".").glob("churn_clean*.csv")]
    print(f"All available cleaned churn csv files: {csvFiles}")

    if not csvFiles:
        print("No churn_cleaned CSV files found...")
        return None
    count = 0
    for fileName in csvFiles:
        if count == 0:
            churnCombinedDf = spark.read.csv(fileName, header=True, inferSchema=True)
            count += 1
        else:
            newChurnDf = spark.read.csv(fileName, header=True, inferSchema=True)
            churnCombinedDf = churnCombinedDf.unionByName(newChurnDf)
    return churnCombinedDf

In [446]:
def get_churn_metadata(churnDataFrame: DataFrame) -> None:
    """
    Displays basic metadata about the churn DataFrame, including data types and non-null counts.
    Converts the Spark DataFrame to a Pandas DataFrame for cleaner output.
    Args:
        churnDataFrame: Spark DataFrame containing the churn dataset.
    Returns:
        None. Prints metadata to the console.
    """
    tempPandasDf = churnDataFrame.toPandas() 
    tempPandasDf.info() 

In [447]:
def split_dataframe_by_label(churnDataFrame: DataFrame) -> tuple[DataFrame, DataFrame, bool]:
    """
    Splits DataFrame into labeled and unlabeled sets based on if churn_risk_score contains empty rows.
    If no unlabeled data exists, reserves 1% of labeled data for demonstrative prediction purposes.
    Args:
        churnDataFrame: Input DataFrame containing cleaned churn data.
    Returns:
        churnlabeled: Rows with churn_risk_score.
        churnunlabeled: Rows without churn_risk_score or reserved data.
        reservedFlag: Boolean True if data was reserved.
    """
    churnlabeled = churnDataFrame.filter(col('churn_risk_score').isNotNull())
    churnunlabeled = churnDataFrame.filter(col('churn_risk_score').isNull())
    reserve = 0.01
    reservedFlag = False
    if churnunlabeled.count() == 0:
        reservedFlag = True
        print('no empty churn_risk_score rows...')
        print(f"Reserving {int(1)}% of labeled data for demo predictions...")
        trainDf, reservedDf = churnlabeled.randomSplit([1 - reserve, reserve], seed=42)
        churnlabeled = trainDf
        churnunlabeled = reservedDf
        print(f"Reserve dataframe with {churnunlabeled.count()} rows")
        churnunlabeled = churnunlabeled.withColumnRenamed("churn_risk_score", "actual_churn_risk_score")
        churnunlabeled = churnunlabeled.withColumn("predicted_churn_risk_score", lit(None))
    return churnlabeled, churnunlabeled, reservedFlag

In [448]:
def separate_feature_target_cols(churnDataFrame: DataFrame) -> tuple[list,str]:
    """
    Separates the input DataFrame columns into features and target variable.
    Uses all columns except the last as features, and the last column as target.
    Args:
        churnDataFrame: Input DataFrame containing cleaned churn data.
    Returns:
        featureCols: List of all feature column names.
        targetCol: Name of the target column to predict.
    """
    featureCols = churnDataFrame.columns[:-1]
    targetCol = churnDataFrame.columns[-1]
    print(f"All Feature Columns: {featureCols}")
    print(f"Target Column: {targetCol}")
    return featureCols, targetCol

In [449]:
def correlation_analysis(churnDataFrame: DataFrame, featureCols: list,
                         targetCol: str, moderateVal: float, strongVal: float) -> list:
    """
    Performs correlation analysis between features and target variable.
    Identifies features with strong, moderate and weak correlation based on threshold values.
    Args:
        churnDataFrame: Input DataFrame containing features and target.
        featureCols: List of feature column names to analyze.
        targetCol: Name of the target column.
        moderateVal: Threshold for moderate correlation.
        strongVal: Threshold for strong correlation.
    Returns:
        list: Features showing strong/moderate correlation with target.
    """
    print("Begin correlation analysis of all features...\n")

    corrVals = []
    for feature in featureCols:
        corr = churnDataFrame.stat.corr(feature, targetCol)
        corrVals.append((feature, corr))

    corrDf = spark.createDataFrame(corrVals, ["feature", "pearson_correlation"]) \
                  .orderBy(abs(col("pearson_correlation")).desc())
    corrDf = corrDf.withColumn("feature_strength",
         when(abs(col("pearson_correlation")) > strongVal, "Strong")
        .when(abs(col("pearson_correlation")) > moderateVal, "Moderate")
        .otherwise("Weak"))
    
    print("\nAll Correlation Scores:")
    corrDf.show(n=corrDf.count(), truncate=False)
    print()
    print("\nFeatures displaying the strongest predictive signal:")
    filteredCorrDf = corrDf.filter(col("feature_strength").isin("Strong", "Moderate"))
    filteredCorrDf.orderBy(abs(col("pearson_correlation")).desc()).show(truncate=False)
    strongCorrFeatures = filteredCorrDf.select("feature").rdd.flatMap(lambda x: x).collect()
    return strongCorrFeatures

In [450]:
def feature_assemble(churnDataframe, featureCols, 
                     trainSplit:float, testSplit:float) -> tuple[DataFrame, DataFrame, bool]:
    """
    Combines features into vector format and splits data into training and test sets.
    Creates feature vectors for XGBoost and performs train-test split as specified.
    Args:
        churnDataframe: Input DataFrame containing features
        featureCols: List of feature column names to assemble
        trainSplit: Proportion of data for training (%)
        testSplit: Proportion of data for testing (%)
    Returns:
        trainDf: Training dataset with feature vectors
        testDf: Test dataset with feature vectors
    """
    print("Assembling features...")
    print("Combining all features into single vector...")
    assembler_all = VectorAssembler(inputCols = featureCols, outputCol="features") 
    pipeline = Pipeline(stages=[assembler_all])
    vectorDf = pipeline.fit(churnDataframe).transform(churnDataframe)
    print("Splitting data...")
    print(f"{trainSplit*100}/{testSplit*100} Test Train Split...")
    trainDf, testDf = vectorDf.randomSplit([trainSplit, testSplit], seed=42)
    return trainDf, testDf

In [451]:
def baseline_model(trainDf, testDf, targetCol) -> tuple[DataFrame, str]:
    """
    Trains and evaluates a baseline XGBoost model using all available features.
    Returns the trained model and its F1 score on the test set.
    Args:
        trainDf: Training DataFrame containing feature vectors
        testDf: Test DataFrame for model evaluation
        targetCol: Name of the target/label column
    Returns:
        xgbModel: Trained XGBoost model
        baselineF1Score: F1 score achieved on test data
    """
    xgbClassifier = SparkXGBClassifier(features_col="features", label_col=targetCol, 
                                       prediction_col="prediction", num_workers=cores)
    xgbModel = xgbClassifier.fit(trainDf)
    predictions = xgbModel.transform(testDf)
    
    evaluator = MulticlassClassificationEvaluator(labelCol=targetCol,predictionCol="prediction",metricName="f1")
    baseF1Score = round(evaluator.evaluate(predictions),5)
    precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
    recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
    acc = MulticlassClassificationEvaluator(labelCol=targetCol,predictionCol="prediction",metricName="accuracy").evaluate(predictions)
    auc = BinaryClassificationEvaluator(labelCol=targetCol,rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(predictions)
    conf_matrix = predictions.groupBy("prediction", "churn_risk_score").count()

    print(f"Base Model F1 Score: {round(baseF1Score, 5)}")
    print(f"Base Model Precision: {round(precision, 5)}")
    print(f"Base Model Recall: {round(recall, 5)}")
    print(f"Accuracy: {round(acc, 5)}")
    print(f"ROC AUC: {round(auc, 5)}")
    conf_matrix.show()
    return xgbModel, baseF1Score

In [452]:
def feature_information_gain(model, featureCols) -> list:
    """
    Extracts and displays feature importances based on information gain from a trained XGBoost model.
    Returns a list of features with strong or moderate importance.
    Args:
        model: Trained XGBoost model from SparkXGBClassifier
        featureCols: List of original feature column names used for training
    Returns:
        strongGainFeatures: List of feature names with 'Strong' or 'Moderate' information gain
    """
    importances = model.get_booster().get_score(importance_type="gain")
    print("All Feature Importances by Information Gain:")

    featureMap = {f"f{i}": name for i, name in enumerate(featureCols)}
    sortedImportances = sorted(importances.items(), key=lambda x: x[1], reverse=True)

    featureDict = {}
    for feature, score in sortedImportances:
        feature = featureMap.get(feature, feature)
        featureDict[feature] = score
    scoreList = [[featureMap.get(feat, feat), score] for feat, score in sortedImportances]
    allFeatureGainDf = spark.createDataFrame(scoreList, ["feature", "gain"])
    allFeatureGainDf = allFeatureGainDf.withColumn("gain_strength",
         when(abs(col("gain")) > 30, "Strong")
        .when(abs(col("gain")) > 10, "Moderate")
        .otherwise("Weak"))
    allFeatureGainDf.show(n=allFeatureGainDf.count(), truncate=False)
    print("Features with stronger information gain scores...")
    strongGainDf = allFeatureGainDf.filter(col("gain_strength").isin("Strong", "Moderate"))
    strongGainDf.orderBy(abs(col("gain_strength")).desc()).show(truncate=False)
    strongGainFeatures = strongGainDf.select("feature").rdd.flatMap(lambda x: x).collect()
    return strongGainFeatures

In [453]:
def get_strong_features(corrFeatures: list, gainFeatures: list) -> list:
    """
    Combines features selected via correlation and information gain into a unified list of strong features.
    Args:
        corrFeatures: List of features selected based on correlation analysis
        gainFeatures: List of features selected based on information gain from the model
    Returns:
        strongFeatures: Combined list of unique features deemed strong by either method
    """
    strongFeatures = list(set(corrFeatures + gainFeatures))
    print(f"Strong feature list: {strongFeatures}")
    return strongFeatures

In [454]:
def kfold_cross_validation(trainDf, testDf, featureCols, 
                           targetCol, k=2)-> tuple[GBTClassificationModel, float]:
    """
    Performs k-fold cross-validation using a Gradient-Boosted Tree (GBT) model and evaluates performance on test data.
    Args:
        trainDf: Training DataFrame containing feature columns and target
        testDf: Test DataFrame for final model evaluation
        featureCols: List of selected feature column names
        TargetCol: Name of the target/label column
        modelType: Type of model to train (currently supports "GBT" only)
        k: Number of folds for cross-validation
    Returns:
        bestModel: Best-performing GBT model from cross-validation
        optimisedF1Score: F1 score of the best model evaluated on the test set
    """
    print(f"Running {k}-fold CV on training data")
    assembler = VectorAssembler(inputCols= featureCols, outputCol="strongFeatures")
    train = assembler.transform(trainDf).select("strongFeatures", targetCol)
    test = assembler.transform(testDf).select("strongFeatures", targetCol)

    model = SparkXGBClassifier(label_col=targetCol, 
                               features_col="strongFeatures",
                               prediction_col="prediction",
                               num_workers=cores)
    paramGrid = ParamGridBuilder() \
        .addGrid(model.getParam("max_depth"), [3, 5]) \
        .addGrid(model.getParam("learning_rate"), [0.05, 0.1]) \
        .addGrid(model.getParam("subsample"), [0.8, 1.0]) \
        .build()

    # Evaluator
    evaluator = MulticlassClassificationEvaluator(
        labelCol=targetCol,
        predictionCol="prediction",
        metricName="f1")

    # CrossValidator
    cv = CrossValidator(
        estimator=model,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        numFolds=k,
        parallelism=cores)

    cvModel = cv.fit(train)
    predictions = cvModel.transform(test)
    print(predictions)
    optimisedF1Score = round(evaluator.evaluate(predictions),5)
    precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
    recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
    
    acc = MulticlassClassificationEvaluator(labelCol=targetCol,predictionCol="prediction",metricName="accuracy").evaluate(predictions)
    auc = BinaryClassificationEvaluator(labelCol=targetCol,rawPredictionCol="rawPrediction", metricName="areaUnderROC").evaluate(predictions)
    conf_matrix = predictions.groupBy("prediction", "churn_risk_score").count()

    print(f"Optimised Model F1 Score: {round(optimisedF1Score, 5)}")
    print(f"Optimised Precision: {round(precision, 5)}")
    print(f"Optimised Recall: {round(recall, 5)}\n")
    print(f"Accuracy: {round(acc, 5)}")
    print(f"ROC AUC: {round(auc, 5)}")
    conf_matrix.show()
    
    return cvModel.bestModel, optimisedF1Score

In [455]:
def select_best_model(baselineF1, optimisedF1, baseModel, 
                      optimisedModel, strongFeatures, allFeatures) -> tuple[ClassificationModel, str, list]:
    """
    Compares baseline and optimised model F1 scores and selects the better-performing model.
    Args:
        baselineF1: F1 score of the baseline model
        optimisedF1: F1 score of the optimised model
        baseModel: Trained baseline model
        optimisedModel: Trained optimised model from cross-validation
        strongFeatures: List of features used in the optimised model
        allFeatures: List of features used in the baseline model
    Returns:
        model: The selected model (either baseline or optimised)
        modelType: String indicating which model was selected ("base" or "optimised")
        features: List of features associated with the selected model
    """
    print(f"Base model F1 Score: {baselineF1}")
    print(f"Optimised model F1 Score: {optimisedF1}")
    if optimisedF1 > baselineF1:
        model = optimisedModel
        modelType = "optimised"
        features = strongFeatures
        print(f"Optimised model selected...")
    else:
        model = baseModel
        modelType = "base"
        features = allFeatures
        print(f"Base model selected...")
    return model, modelType, features

In [456]:
def predict_churn_score(modelType, model, 
                        churnDataUnlabelled: DataFrame, features, reservedFlag) -> None:
    """
    Applies the selected model to unlabeled churn data and displays predictions along with accuracy if labels exist.
    Args:
        modelType: Type of model used ("base" or "optimised")
        model: Trained classification model
        churnDataUnlabelled: DataFrame containing data to predict churn scores on
        features: List of feature columns to be used for prediction
        reservedFlag: Reserved argument (not currently used)
    Returns:
        None. Prints prediction results and accuracy (if actual labels are available).
    """
    if modelType == "optimised":
        feature_vector_col = "strongFeatures"
        selectedFeatures = ['actual_churn_risk_score'] + features
    else:
        feature_vector_col = "features"
        selectedFeatures = ['actual_churn_risk_score'] + features

    churnDataUnlabelled = churnDataUnlabelled.select(*[feature for feature in selectedFeatures if feature in churnDataUnlabelled.columns])

    assembler = VectorAssembler(inputCols=features, outputCol=feature_vector_col)
    churnunlabeledPredict = assembler.transform(churnDataUnlabelled)
    predictions = model.transform(churnunlabeledPredict)

    predictions = predictions.withColumnRenamed("prediction", "predicted_churn_risk_score")
    
    # Determine which label column to use
    if "actual_churn_risk_score" in predictions.columns:
        label_col = "actual_churn_risk_score"
    elif "churn_risk_score" in predictions.columns:
        label_col = "churn_risk_score"

    if label_col:
        selected_df = predictions.select(label_col, "predicted_churn_risk_score")
        print(f"\n----- Prediction Results ({modelType} model using '{label_col}') -----")
        selected_df = selected_df.withColumn("predicted_churn_risk_score", selected_df["predicted_churn_risk_score"].cast(IntegerType()))
        selected_df.show(truncate=False)

        correct = predictions.filter(col(label_col) == col("predicted_churn_risk_score")).count()
        total = predictions.count()
        accuracy = correct / total if total else 0
        print(f"Prediction Accuracy: {accuracy:.2%} ({correct}/{total})")
    else:
        print("\nNo label column found ('actual_churn_risk_score' or 'churn_risk_score') in data.")
        predictions.select("predicted_churn_risk_score").show(truncate=False)

In [457]:
def execute_ml_pipeline() -> None:
    """
    Orchestrates the entire machine learning pipeline for churn prediction.
    This function performs the following steps:
        1. Combines and loads cleaned churn data.
        2. Splits the dataset into labeled and unlabeled data.
        3. Trains a baseline XGBoost model using all features.
        4. Evaluates feature importance via correlation and information gain.
        5. Trains an optimised model using top-ranked features and cross-validation.
        6. Compares baseline and optimised models and selects the better one.
        7. Applies the best model to make predictions on unseen (unlabeled) data.
        8. Logs pipeline execution time.
    Returns:
        None. All results are printed to the console during execution.
    """
    
    startTime = time.time()
    
    print("Spark session created: ChurnXGBoost ")
    print()

    randomId = uuid.uuid4()
    stringID = str(randomId)
    print(f"Executing Machine Learning Pipeline...")
    print() 
    print(f"Run ID: {stringID}")
    print()

    print("***** DATA PREPARATION *****", end="\n")
    print()
    
    print("===== Combine Cleaned Files and Load Data =====", end="\n")
    churnData = concatenate_clean_churn_files()

    print("===== Cleaned Churn Metadata =====", end="\n")
    get_churn_metadata(churnData)
    print()

    print("===== Split Dataframe by labels =====", end="\n")
    churnDataLabelled, churnDataUnlabelled, reservedFlag = split_dataframe_by_label(churnData)
    print()

    print("***** TRAIN AND SCORE BASELINE MODEL *****", end="\n")
    print()

    print("===== Baseline Model: Separate Feature and Target Column/s =====", end="\n")
    allFeatures, targetCol = separate_feature_target_cols(churnDataLabelled)
    print()

    print("===== Baseline Model: Correlation Analysis =====", end="\n")
    strongCorrFeatures = correlation_analysis(churnDataLabelled, allFeatures, targetCol, 0.2, 0.4)
    print()

    print("===== Baseline Model: Assemble Features =====", end="\n")
    trainSetDf, testSetDf = feature_assemble(churnDataLabelled, allFeatures, trainSplit=0.8, testSplit=0.2)
    print()

    print("===== Baseline Model: Create and Score =====",end="\n")
    baseModel, baselineF1Score = baseline_model(trainSetDf, testSetDf, targetCol)
    print()

    print("===== Baseline Model: Feature Information Gain =====",end="\n")
    strongGainFeatures = feature_information_gain(baseModel, allFeatures)
    print()

    print("***** TRAIN AND SCORE OPTIMISED MODEL *****",end="\n")
    print()

    print("===== Optimised Model: Strongest Features =====",end="\n")
    strongFeatures = get_strong_features(strongCorrFeatures, strongGainFeatures)
    print()

    print("===== Optimised Model: K-Fold Cross Validation =====",end="\n")
    optimisedModel, optimisedF1Score = kfold_cross_validation(trainSetDf, testSetDf, strongFeatures, targetCol)
    print()

    print("***** SELECT BEST MODEL AND PREDICT UNSEEN DATA *****",end="\n")
    print()

    print("===== Select Best Model =====",end="\n")
    bestModel, modelType, features = select_best_model(baselineF1Score, optimisedF1Score,
                                             baseModel, optimisedModel, strongFeatures, allFeatures)
    print()

    print("===== Apply Model to Unseen Data =====",end="\n")
    predict_churn_score(modelType, bestModel, churnDataUnlabelled, features, reservedFlag)
    
    endTime = time.time()
    pipelineRunTime = endTime - startTime
    print(f"Machine learning pipeline complete. Run time: {timedelta(seconds=int(pipelineRunTime))}")

In [ ]:
execute_ml_pipeline() # Execute pipeline

Spark session created: ChurnXGBoost 

Executing Machine Learning Pipeline...

Run ID: bbfcf275-36d2-4c50-b086-a498509fa379

***** DATA PREPARATION *****

===== Combine Cleaned Files and Load Data =====
All available cleaned churn csv files: ['churn_clean.csv']
===== Cleaned Churn Metadata =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           20000 non-null  int32  
 1   gender                        20000 non-null  int32  
 2   region_category               20000 non-null  int32  
 3   membership_category           20000 non-null  int32  
 4   joined_through_referral       20000 non-null  int32  
 5   preferred_offer_types         20000 non-null  int32  
 6   medium_of_operation           20000 non-null  int32  
 7   internet_option               20000 non-null  int32  
 8   


***** TRAIN AND SCORE OPTIMISED MODEL *****

===== Optimised Model: Strongest Features =====
Strong feature list: ['membership_category', 'points_in_wallet', 'avg_transaction_value']

===== Optimised Model: K-Fold Cross Validation =====
Running 2-fold CV on training data
